# Named Entity Linking

The goal is, given an article, to find all tokens (or groups of tokens) that describe the same physical person.

## Imports

In [1]:
import spacy
from spacy.tokens import Span
import xml.etree.ElementTree as ET

In [2]:
# Loading language model
nlp = spacy.load("fr_core_news_md")

## Data

Loading a test article, which has the following content:

Comment les chouettes effraies au plumage blanc, particulièrement visibles la nuit, parviennent-elles à attraper des proies? L’énigme était cachée dans les cycles de la lune et dans un curieux comportement de ses proies, révèle une étude lausannoise.

Les nuits de pleine lune, tous les sortilèges sont de mise. Les loups-garous se déchaînent; les vampires se régénèrent; et les jeunes filles se muent en sirènes. Même les chouettes effraies (Tyto alba) sont de la partie. Comme chaque nuit, elles partent en chasse. Mais "les nuits de pleine lune, les plus claires d’entre elles resplendissent comme des soleils", observe __Alexandre Roulin__, de l’Université de Lausanne. Comme camouflage vis-à-vis des proies, il y a mieux!

Au vrai, la chouette effraie présente, d’un individu à l’autre, une grande variété de plumages. Certains individus sont presque entièrement vêtus de roux; d’autres, de blanc (tous les intermédiaires sont possibles). Comment les chouettes les plus blanches, visibles comme le loup blanc les nuits de pleine lune, ont-elles pu survivre? Pourquoi n’ont-elles pas été supplantées par leurs congénères rousses, bien plus discrètes dans le noir? Pourquoi les lois de l’évolution les ont-elles épargnées, alors qu’elles éliminent sans pitié les plus faibles ou les moins performants?

La réponse a été publiée dans la revue Nature Ecology & Evolution. Les auteurs ont exploité leur base de données, accumulées depuis trente ans en Suisse par l’équipe d’ __Alexandre Roulin__. "Nous avons installé 400 nichoirs contre des granges et suivi le devenir de plus de 1000 couvées: œufs pondus, poids des nichées, proies rapportées chaque nuit par le mâle – qui est le seul à chasser…" Puis les auteurs ont confronté ces données au cycle lunaire, nuit après nuit.

A l’aide de caméras infrarouges, ils ont d’abord montré que les mâles rapportaient au nid, en moyenne, 4,78 proies par nuit. Mais ce nombre variait selon la couleur du plumage… et le cycle lunaire. Ainsi, les mâles les plus roux capturaient 5,67 proies les nuits de nouvelle lune, contre 3,27 seulement les nuits de pleine lune. Et les mâles les plus blancs? "Contre toute attente, leurs performances n’étaient pas affectées par les nuits de pleine lune", souligne __Luis San-Jose__, premier auteur de l’étude. Ils attrapaient ainsi 4,94 proies les nuits de nouvelle lune, contre 4,61 les nuits de pleine lune – une différence non significative.

Ensuite, les chercheurs ont équipé les oiseaux mâles de balises GPS pour évaluer leurs succès de chasse. Résultats: quand l’effort de chasse était inférieur à la moyenne (peu de tentatives effectuées), ni la couleur du plumage, ni le cycle lunaire n’avaient d’effet. Mais quand l’effort de chasse était soutenu, tout changeait. Les mâles les plus roux voyaient 48% de leurs tentatives couronnées de succès les nuits de nouvelle lune; contre 42% les nuits de pleine lune. Mais pour les mâles les plus blancs, le cycle lunaire n’avait aucun effet sur leur taux de succès – autour de 42%.

A quoi attribuer ces différences? Fallait-il regarder du côté des principales proies de cette chouette, des rongeurs? "Nous avons capturé des campagnols des champs, que nous avons placés trois jours en cage, en faisant varier l’éclairage pour mimer les différentes phases de la lune. Dans le même temps, nous avons fait "voler" au-dessus de leurs têtes, à l’aide de tyroliennes, des chouettes empaillées de différentes couleurs", raconte __Alexandre Roulin__. Résultat: les "nuits de pleine lune", quand la chouette était blanche, ses proies potentielles se figeaient deux fois plus longtemps que les "nuits de nouvelle lune". Un effet inexistant pour les chouettes rousses.

"Le plumage blanc réfléchit la lumière de la lune. Pour les rongeurs, c’est comme s’ils recevaient en pleine face la lumière du soleil reflétée par un miroir. Stressés, ils s’immobilisent. Et les chouettes blanches ont plus de facilité pour les attraper", explique le chercheur. En somme, la chouette effraie blanche effraie mieux ses proies: elle exploite l’aversion naturelle des rongeurs pour la lumière brillante. Nul maléfice, donc, dans la couleur de cette "dame blanche", mais une stratégie de chasse efficace. Pourtant, cette chouette continue d’effrayer les humains. Visage blanc, vol furtif, chuintements bizarres: il n’en fallait pas plus pour faire de cet oiseau de nuit un avatar de fantôme. Dans nos campagnes, elle fut longtemps clouée aux portes des granges pour conjurer le mauvais sort. En tant que grande consommatrice de rongeurs, elle est pourtant un des meilleurs alliés de l’homme.

In [3]:
test_article_path = '../data/article01.xml'
with open(test_article_path) as f:
    test_article_xml = f.read()

In [4]:
QUOTES = ["«", "»", "“", "”", "„", "‹", "›", "‟", "〝", "〞"]

def normalize_quotes(text, default_quote='"', quotes=None):
    if quotes is None:
        quotes = QUOTES
    for q in quotes:
        text = text.replace(q, default_quote)
    return text

def get_element_text(el):
    ls = list(el.itertext())
    text = ''.join(ls).replace('\n', '')
    text = ' '.join(text.split())
    return normalize_quotes(text)

def extract_paragraphs(root):
    elements = root.findall('p')
    return [get_element_text(el) for el in elements]

root = ET.fromstring(test_article_xml)
paragraphs = extract_paragraphs(root)
test_article = '\n'.join(paragraphs)

In [5]:
doc = nlp(test_article)

## Finding all Named Entites that are people

### Initial evaluation

We expect the output to be:
* Alexandre Roulin
* Alexandre Roulin
* Luis San-Jose
* Alexandre Roulin

In [6]:
for ent in doc.ents:
    if ent.label_ == 'PER':
        print(ent.text)

Tyto
Alexandre Roulin
Alexandre Roulin
Luis San
Jose
Alexandre Roulin
Stressés


We notice a few things:
* The spaCy models sees "Tyto" and "Stressés" as names, which we can't do much about.
* Luis San-Jose has his name cut in two due to the hyphen. This we can correct.

### Fixing hyphens

We fix hyphens by adding a custom rule: if two named entities that are of the "person" type are seperated by a unique hypen, they are merged together.

In [7]:
def merge_hyphen_NEs(ents):    
    corrected_ents = []
    prev_per_ent = None
    for ent in ents:
        if ent.label_ == 'PER':
            if (prev_per_ent is not None) and \
            (prev_per_ent.end_char == ent.start_char - 1) and \
            (doc.text[prev_per_ent.end_char] == '-'):
                # Create a new NE for the name with the hyphen
                merged_ent = Span(doc, prev_per_ent.start, ent.end, label="PER")
                # Remove the last entity and add the new one.
                corrected_ents[-1] = merged_ent
            else:
                corrected_ents.append(ent)
            prev_per_ent = ent
        else:
            corrected_ents.append(ent)
            prev_per_ent = None
    return corrected_ents

doc.ents = merge_hyphen_NEs(doc.ents)
for ent in doc.ents:
    if ent.label_ == 'PER':
        print(ent.text)

Tyto
Alexandre Roulin
Alexandre Roulin
Luis San-Jose
Alexandre Roulin
Stressés


## Grouping Names

We want to group the different ways that someone can be mentioned into unique entities.

Each person NE in the document is mapped to the longest form of the name of the person.

In [14]:
all_names = ['Alexandre', 'Roulin', 'Alexandre Roulin', 'Luis', 'San-Jose', 'Luis San-Jose']

In [15]:
all_names.sort(key=lambda x: -len(x.split(' ')))
print(all_names)

['Alexandre Roulin', 'Luis San-Jose', 'Alexandre', 'Roulin', 'Luis', 'San-Jose']


In [37]:
people = set()
full_names = {}

def is_substring(person, name):
    person_tokens = person.split(' ')
    name_tokens = name.split(' ')
    for i in range(len(person_tokens)):
        matching_tokens = 0
        j = 0
        while (i + j) < len(person_tokens) and \
                j < len(name_tokens) and \
                person_tokens[i + j] == name_tokens[j]:
            j = j + 1
        if j == len(name_tokens):
            return True
    return False
        

def find_full_name(all_people, name):
    for person in all_people:
        if is_substring(person, name):
            return person
    return name

for name in all_names:
    if name not in people:
        full_name = find_full_name(people, name)
        full_names[name] = full_name
        if name == full_name:
            people.add(name)

print('people found:', people, '\n')
for name in full_names:
    print(f'Name:      {name}\nFull Name: {full_names[name]}\n')

people found: {'Luis San-Jose', 'Alexandre Roulin'} 

Name:      Alexandre Roulin
Full Name: Alexandre Roulin

Name:      Luis San-Jose
Full Name: Luis San-Jose

Name:      Alexandre
Full Name: Alexandre Roulin

Name:      Roulin
Full Name: Alexandre Roulin

Name:      Luis
Full Name: Luis San-Jose

Name:      San-Jose
Full Name: Luis San-Jose



## Possible Improvements

* Linking elements such as A. Roulin to Alexandre Roulin.
* Distinguishing between people with the same last name.